In [1]:
using Laplacians

objc[32047]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[32047]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[32047]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[32047]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("../../../src/pcgMatNorm.jl")
include("../../../src/fastCSC.jl")
include("../../../src/KMPSolver.jl")
include("../../../src/samplingSolver.jl")

checkError (generic function with 1 method)

In [3]:
n = 100000;
a = wtedChimera(n,23); la = lap(a) + speye(n) * 1e-15;
truex = exp(randn(n)); b = la * truex;

### Trying out augTreeSolver

In [4]:
F = augTreePrecon(la, treeAlg=akpw);

In [5]:
function fat(b; tol=1e-4, maxits=Inf, maxtime=Inf, verbose=false)
    return pcg(la, b, F, tol=tol, maxits=maxits, maxtime=maxtime, verbose=verbose)
end

fat (generic function with 1 method)

In [6]:
@time fat(b, verbose=true);

PCG stopped after: 118 iterations with relative error 6.982909171149705e-5.
  2.140369 seconds (608.51 k allocations: 928.486 MB, 7.30% gc time)


In [7]:
function fatMN(b; tol=1e-4, maxits=Inf, maxtime=Inf, verbose=false)
    return pcgMatNorm(la, b, F, truex, tol=tol, maxits=maxits, maxtime=maxtime, verbose=verbose)
end

fatMN (generic function with 1 method)

In [8]:
@time fatMN(b, verbose=true);

PCG stopped after: 777 iterations with relative error 9.908102470601774e-5.
 14.251148 seconds (465.89 k allocations: 8.129 GB, 8.06% gc time)


### Trying for KMP

In [9]:
params = KMPparams(1/36, 6, 0.125, 600, :akpw);

a2 = copy(a);
tree = akpw(a2);
ord = Laplacians.dfsOrder(tree);
aord = symPermuteCSC(a2,ord);
tord = symPermuteCSC(tree,ord)
la = lap(aord);
fsub = KMPLapPrecon(aord, tord, params, verbose=true);

aveStretch : 5.378693867716532 fac : 11.167271420603699
level 0. Dimension 100000 off-tree edges : 237763
level 1. Dimension 22069 off-tree edges : 8371


In [10]:
function fkmp(b::Array{Float64,1}; tol=1e-4, maxits=Inf, verbose=false)
    bord = b[ord]

    xord = pcg(la, bord, fsub, tol=tol, maxits=maxits, verbose=verbose)

    x = zeros(Float64,n)
    x[ord] = xord
    subMean!(x)
    return x
end

fkmp (generic function with 1 method)

In [11]:
@time fkmp(b, verbose=true);

PCG stopped after: 95 iterations with relative error 8.832949531571389e-5.
  2.638736 seconds (274.53 k allocations: 1.915 GB, 9.52% gc time)


In [12]:
function fkmpMN(b::Array{Float64,1}; tol=1e-4, maxits=Inf, verbose=false)
    bord = b[ord]

    xord = pcgMatNorm(la, bord, fsub, truex[ord], tol=tol, maxits=maxits, verbose=verbose)

    x = zeros(Float64,n)
    x[ord] = xord
    subMean!(x)
    return x
end

fkmpMN (generic function with 1 method)

In [13]:
@time fkmpMN(b, verbose=true);

PCG stopped after: 317 iterations with relative error 9.964371557888412e-5.
  9.572105 seconds (243.65 k allocations: 7.301 GB, 10.12% gc time)


### Sampling Solver

In [14]:
@time F,gOp,U,d,ord,cn,cntime = buildSolver(a, eps=0.5, sampConst=0.02, beta=1e3, verbose=false);
checkError(gOp)

  3.857845 seconds (12.33 M allocations: 1.103 GB, 11.22% gc time)


0.9004205343191517

In [15]:
la = lap(symPermuteCSC(a, ord));

In [16]:
function fss(b)
    auxb = copy(b)
    push!(auxb, -sum(auxb))

    ret = pcg(la, auxb[ord], F, tol=1e-4, maxits=Inf, maxtime=Inf, verbose=true)
    ret = ret[invperm(ord)]

    # We want to discard the nth element of ret (which corresponds to the first element in the permutation)
    ret = ret - ret[n]

    pop!(ret)

    return ret
end

fss (generic function with 1 method)

In [17]:
@time fss(b);

PCG stopped after: 130 iterations with relative error 9.557084200008583e-5.
  2.326338 seconds (26.02 M allocations: 1.168 GB, 7.89% gc time)


In [18]:
function fssMN(b)
    auxb = copy(b)
    push!(auxb, -sum(auxb))

    ret = pcgMatNorm(la, auxb[ord], F, truex[ord], tol=1e-4, maxits=Inf, maxtime=Inf, verbose=true)
    ret = ret[invperm(ord)]

    # We want to discard the nth element of ret (which corresponds to the first element in the permutation)
    ret = ret - ret[n]

    pop!(ret)

    return ret
end

fssMN (generic function with 1 method)

In [19]:
@time fssMN(b);

PCG stopped after: 269 iterations with relative error 9.964456706126148e-5.
  6.270604 seconds (53.81 M allocations: 3.216 GB, 8.24% gc time)
